선형회귀 시도, 변수선택법 적용  
mae = 1.559

## 1. EDA

In [124]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [125]:
# 1. 데이터 로드
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")
submission_df = pd.read_csv("data/sample_submission.csv")

# test.csv에서 원본 id 저장
test_ids = test_df["id"].copy()
test_df = test_df.drop(columns=["id"])  # 이후 분석을 위해 삭제

In [126]:
train_df.describe()

,id,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,7499.500000,1.316742,1.023813,0.347326,23.372701,10.104499,5.040622,6.704216,9.966800
std,4330.271354,0.287869,0.237697,0.091335,12.754705,5.691158,2.805236,3.598253,3.238065
min,0.000000,0.187500,0.150000,0.000000,0.056699,0.028349,0.014175,0.042524,1.000000
25%,3749.750000,1.150000,0.887500,0.287500,13.377420,5.698250,2.820775,3.827183,8.000000
50%,7499.500000,1.375000,1.075000,0.362500,23.657658,9.879801,4.904464,6.803880,10.000000
75%,11249.250000,1.537500,1.200000,0.412500,32.205032,14.033003,7.002326,9.071840,11.000000
max,14999.000000,1.950000,1.575000,0.700000,80.101512,48.477645,19.220961,24.564842,29.000000


## 2. 데이터 전처리

In [127]:
# 'Sex' 라벨 인코딩
label_encoder = LabelEncoder()
train_df["Sex"] = label_encoder.fit_transform(train_df["Sex"])
test_df["Sex"] = label_encoder.transform(test_df["Sex"])

In [128]:
# Height가 0인 경우 평균값으로 대체
height_mean = train_df.loc[train_df["Height"] > 0, "Height"].mean()
train_df.loc[train_df["Height"] == 0, "Height"] = height_mean
test_df.loc[test_df["Height"] == 0, "Height"] = height_mean

In [129]:
# IQR 기반 이상치 제거 함수 정의 및 적용
def remove_outliers_iqr(df, cols, threshold=1.5):  # threshold=3.0에서 변경
    Q1 = df[cols].quantile(0.25)
    Q3 = df[cols].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    return df[~((df[cols] < lower_bound) | (df[cols] > upper_bound)).any(axis=1)]

num_cols = train_df.select_dtypes(include=["float64"]).columns
train_df = remove_outliers_iqr(train_df, num_cols, threshold=1.5)

In [130]:
# 중복 데이터 제거 및 'id' 컬럼 삭제
train_df = train_df.drop_duplicates().drop(columns=["id"])

#### X, y 분리 / train-valid 분리

In [131]:
# X, y 분리
X = train_df.drop(columns=["Age"])
y = train_df["Age"]

In [132]:
# Train/Validation Split (8:2 비율)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [133]:
# 3. 스케일링
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

## 3. 머신러닝

In [134]:
def separate_train(df): # 'count' 피쳐를 y로 구분
    X = df.drop(['Age', 'Sex'], axis=1)
    y = df['Age']
    return X, y

In [135]:
# 4. 선형회귀 1
X, y = separate_train(train_df)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_prediction = model.predict(X_valid)
mean_absolute_error(y_valid, y_prediction)

1.1451920891579899

#### 변수 선택

In [136]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error

from sklearn.feature_selection import SequentialFeatureSelector as SFS

# 5. Cross Validation 설정 (KFold 사용)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

In [137]:
# MAE를 가장 작게 하는 변수 선택(전진선택법)
sfs = SFS(
    model,
    n_features_to_select="auto",  # 자동으로 최적의 변수 개수 선택(auto) - best는 안 됨|
    direction="backward",  # 전진 선택법 ("backward"로 하면 후진 제거법)
    scoring="neg_mean_absolute_error",
    cv=5,
    n_jobs=-1
)

# 훈련 데이터로 속성(변수) 선택 실행
sfs = sfs.fit(X_train_scaled, y_train)

# 선택된 변수 인덱스 가져오기
selected_idx = sfs.get_support(indices=True)

# 선택된 변수 인덱스
print("선택된 변수 인덱스:", selected_idx)

선택된 변수 인덱스: [2 3 5 7]


- `model` : 주어진 모델을 이용해 학습을 진행한다는 뜻. SequentialFeatureSelector의 입력으로 제공됨.

- `n_features_to_select` : 선택할 변수(특성)의 개수 지정
    - auto: 자동으로 최적의 변수 개수 선택
    - int값(정수)을 주면 그 숫자만큼 특성 선택

- `direction` : 특성 선택 방법을 결정
    - forward: 전진선택법
    - backward: 후진선택법

- `scoring` : 모델 평가 시 사용할 성능지표 지정
    - `neg_mean_absolute_error` : MAE(평균절대오차) 기준으로 성능 평가.
    - SFS는 변수 조합에서 `scoring`값이 최대가 되는 특성 조합 선택.
    - 다른 성능 지표로도 사용될 수 있고, 회귀에서는 `neg_mean_squared_error`, 분류에서는 `accuracy`를 주로 사용.

- `cv` : 교차검증을 위한 KFold 분할 수 지정.
    - `cv=5` : 데이터를 5개 폴드로 나눠서 5개의 교차검증 진행.

In [138]:
# 선택된 변수만을 사용하여 선형 회귀 모델 다시 학습
X_train_selected = X_train_scaled[:, selected_idx]  # 선택된 변수만 사용
X_valid_selected = X_valid_scaled[:, selected_idx]  # 검증 데이터도 동일하게 변환

# 새로운 선형 회귀 모델 학습
model.fit(X_train_selected, y_train)

# 검증 데이터 예측 및 MAE 계산
y_pred = model.predict(X_valid_selected)
mae = mean_absolute_error(y_valid, y_pred)
print(f"Validation MAE (선택된 변수 사용): {mae:.4f}")

Validation MAE (선택된 변수 사용): 1.1559
